In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")


In [7]:
ordersDf = spark.read.format("json")\
                        .option("inferSchema", "true")\
                         .option("multiLine", "true")\
                         .load(f"{work_dir}/test.json")
display(ordersDf)

datasets,filename
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails


In [11]:
from pyspark.sql.functions import explode
parseOrdersDf = ordersDf.withColumn("orders", explode(ordersDf.datasets))
display(parseOrdersDf)

datasets,filename,orders
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240, ..."
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5002, 2021-12-25 00.00.00.000, [{prd9001, 1, 1, {275, 250, 25}}, {prd9004, 4, 2, {1000, 900,..."


In [15]:
# Step 3: Fetch Each Order using getItem on explode column
parseOrdersDf = parseOrdersDf.withColumn("customerId", parseOrdersDf.orders.getItem("customerId"))\
                             .withColumn("orderId", parseOrdersDf.orders.getItem("orderId"))\
                             .withColumn("orderDate", parseOrdersDf.orders.getItem("orderDate"))\
                             .withColumn("orderDetails", parseOrdersDf.orders.getItem("orderDetails"))\
                             .withColumn("shipmentDetails", parseOrdersDf.orders.getItem("shipmentDetails"))
display(parseOrdersDf)

datasets,filename,orders,customerId,orderId,orderDate,orderDetails,shipmentDetails
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240, ...",cust5001,ord1001,2021-12-24 00.00.00.000,"[{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240, 60}}]","{Delhi, India, 110040, New Delhi, M.G.Road}"
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5002, 2021-12-25 00.00.00.000, [{prd9001, 1, 1, {275, 250, 25}}, {prd9004, 4, 2, {1000, 900,...",cust5002,ord1002,2021-12-25 00.00.00.000,"[{prd9001, 1, 1, {275, 250, 25}}, {prd9004, 4, 2, {1000, 900, 100}}]","{Mumbai, India, 400064, Maharastra, Malad}"


In [16]:
parseOrdersDf = parseOrdersDf.withColumn("orderDetails", explode(parseOrdersDf.orderDetails))
display(parseOrdersDf)

datasets,filename,orders,customerId,orderId,orderDate,orderDetails,shipmentDetails
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240, ...",cust5001,ord1001,2021-12-24 00.00.00.000,"{prd9001, 2, 1, {550, 500, 50}}","{Delhi, India, 110040, New Delhi, M.G.Road}"
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240, ...",cust5001,ord1001,2021-12-24 00.00.00.000,"{prd9002, 3, 2, {300, 240, 60}}","{Delhi, India, 110040, New Delhi, M.G.Road}"
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5002, 2021-12-25 00.00.00.000, [{prd9001, 1, 1, {275, 250, 25}}, {prd9004, 4, 2, {1000, 900,...",cust5002,ord1002,2021-12-25 00.00.00.000,"{prd9001, 1, 1, {275, 250, 25}}","{Mumbai, India, 400064, Maharastra, Malad}"
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5002, 2021-12-25 00.00.00.000, [{prd9001, 1, 1, {275, 250, 25}}, {prd9004, 4, 2, {1000, 900,...",cust5002,ord1002,2021-12-25 00.00.00.000,"{prd9004, 4, 2, {1000, 900, 100}}","{Mumbai, India, 400064, Maharastra, Malad}"


In [17]:
parseOrdersDf = parseOrdersDf.withColumn("productId", parseOrdersDf.orderDetails.getItem("productId"))\
                             .withColumn("quantity", parseOrdersDf.orderDetails.getItem("quantity"))\
                             .withColumn("sequence", parseOrdersDf.orderDetails.getItem("sequence"))\
                             .withColumn("totalPrice", parseOrdersDf.orderDetails.getItem("totalPrice"))\
                             .withColumn("city", parseOrdersDf.shipmentDetails.getItem("city"))\
                             .withColumn("country", parseOrdersDf.shipmentDetails.getItem("country"))\
                             .withColumn("postalcode", parseOrdersDf.shipmentDetails.getItem("postalCode"))\
                             .withColumn("street", parseOrdersDf.shipmentDetails.getItem("street"))\
                             .withColumn("state", parseOrdersDf.shipmentDetails.getItem("state"))
display(parseOrdersDf)

datasets,filename,orders,customerId,orderId,orderDate,orderDetails,shipmentDetails,productId,quantity,sequence,totalPrice,city,country,postalcode,street,state
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240, ...",cust5001,ord1001,2021-12-24 00.00.00.000,"{prd9001, 2, 1, {550, 500, 50}}","{Delhi, India, 110040, New Delhi, M.G.Road}",prd9001,2,1,"{550, 500, 50}",Delhi,India,110040,M.G.Road,New Delhi
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240, ...",cust5001,ord1001,2021-12-24 00.00.00.000,"{prd9002, 3, 2, {300, 240, 60}}","{Delhi, India, 110040, New Delhi, M.G.Road}",prd9002,3,2,"{300, 240, 60}",Delhi,India,110040,M.G.Road,New Delhi
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5002, 2021-12-25 00.00.00.000, [{prd9001, 1, 1, {275, 250, 25}}, {prd9004, 4, 2, {1000, 900,...",cust5002,ord1002,2021-12-25 00.00.00.000,"{prd9001, 1, 1, {275, 250, 25}}","{Mumbai, India, 400064, Maharastra, Malad}",prd9001,1,1,"{275, 250, 25}",Mumbai,India,400064,Malad,Maharastra
"[{cust5001, 2021-12-24 00.00.00.000, [{prd9001, 2, 1, {550, 500, 50}}, {prd9002, 3, 2, {300, 240,...",orderDetails,"{cust5002, 2021-12-25 00.00.00.000, [{prd9001, 1, 1, {275, 250, 25}}, {prd9004, 4, 2, {1000, 900,...",cust5002,ord1002,2021-12-25 00.00.00.000,"{prd9004, 4, 2, {1000, 900, 100}}","{Mumbai, India, 400064, Maharastra, Malad}",prd9004,4,2,"{1000, 900, 100}",Mumbai,India,400064,Malad,Maharastra
